In [1]:
import numpy as np # linear algebra
import pandas as pd

In [2]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SVM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SVM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SVM\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [9]:
df = pd.read_csv("sqli.csv",encoding='utf-16')
df.tail(10)

,Sentence,Label
4190,pdf 201704/00001491531745287,0
4191,zip 201704/00001491531967789,0
4192,pdf 20170407112157Xyhcho 202 2020 203 204 2043...,0
4193,"com"" style = ""max-width:450px;float:left;paddi...",0
4194,"0"" encoding = ""utf-8"" ?> <_filter> <![CDATA...",0
4195,"org/?option = com_k2 <a href = ""http://corfopym",0
4196,com/?option = com_k2 <act> <![CDATA[procMemb...,0
4197,picsearch,0
4198,com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...,0
4199,de]]> </email_address> <find_account_answer...,0


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()

In [35]:
transformed_posts=pd.DataFrame(posts)
df=pd.concat([df,transformed_posts],axis=1)
print(df.columns)
X=df[df.columns[2:]]
y=df['Label']


Index(['Sentence',    'Label',          0,          1,          2,          3,
                4,          5,          6,          7,
       ...
             4707,       4708,       4709,       4710,       4711,       4712,
             4713,       4714,       4715,       4716],
      dtype='object', length=9436)


In [15]:
y.value_counts()

0    3072
1    1128
Name: Label, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Logistic regression

In [17]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

In [36]:
from sklearn.metrics import accuracy_score, classification_report
y_pred=clf.predict(X_test)
accuracy_score(y_test, y_pred)


0.9285714285714286

In [49]:
#checking how clf predict works on a single sentence
data = ["message "]
vect = vectorizer.transform(data).toarray()
clf.predict(vect)

array([0], dtype=int64)

In [38]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       588
           1       0.98      0.77      0.87       252

    accuracy                           0.93       840
   macro avg       0.95      0.88      0.91       840
weighted avg       0.93      0.93      0.93       840



In [34]:
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,4707,4708,4709,4710,4711,4712,4713,4714,4715,4716
1743,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3337,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Neural Network

In [20]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [22]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                94360     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 110,955
Trainable params: 108,907
Non-trainable params: 2,048
______________________________________________

In [23]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)

Epoch 1/10
224/224 [==============================] - 2s 8ms/step - loss: 0.2193 - accuracy: 0.9060 - val_loss: 0.6864 - val_accuracy: 0.7000
Epoch 2/10
224/224 [==============================] - 1s 4ms/step - loss: 0.1096 - accuracy: 0.9563 - val_loss: 0.4156 - val_accuracy: 0.7095
Epoch 3/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0920 - accuracy: 0.9643 - val_loss: 0.1185 - val_accuracy: 0.9226
Epoch 4/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0908 - accuracy: 0.9676 - val_loss: 0.0661 - val_accuracy: 0.9738
Epoch 5/10
224/224 [==============================] - 1s 3ms/step - loss: 0.0839 - accuracy: 0.9699 - val_loss: 0.0725 - val_accuracy: 0.9750
Epoch 6/10
224/224 [==============================] - 1s 3ms/step - loss: 0.0811 - accuracy: 0.9711 - val_loss: 0.0606 - val_accuracy: 0.9786
Epoch 7/10
224/224 [==============================] - 1s 5ms/step - loss: 0.0887 - accuracy: 0.9682 - val_loss: 0.0771 - val_accuracy: 0.9738
Epoch 

In [42]:
pred=model.predict(X_test)
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [43]:
accuracy_score(y_test,pred)


0.9773809523809524

In [27]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)


In [28]:

accuracy,precision,recall=confusion_matrix(y_test,pred)
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))
from sklearn.metrics import precision_score
precision_score(y_test, pred)


 Accuracy : 0.9773809523809524 
 Precision : 0.9330855018587361 
 Recall : 0.996031746031746


0.9330855018587361

In [40]:
from sklearn.metrics import recall_score
recall_score(y_test, pred)


0.996031746031746

In [41]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       588
           1       0.98      0.77      0.87       252

    accuracy                           0.93       840
   macro avg       0.95      0.88      0.91       840
weighted avg       0.93      0.93      0.93       840



In [45]:
!pip install joblib